In [1]:
from tinygrad.densetensor import DenseTensor
from tinygrad.sparsetensor import SparseTensor
import numpy as np

%load_ext autoreload
%autoreload 2

DEVICE:GPU


In [2]:
x_init = np.random.randn(1,3).astype(np.float32)
x2_init = np.random.randn(3).astype(np.float32)
U_init = np.random.randn(3,3).astype(np.float32)
V_init = np.random.randn(3,3).astype(np.float32)
W_init = np.random.randn(3,3).astype(np.float32)
m_init = np.random.randn(1,3).astype(np.float32)

In [3]:
x = DenseTensor(x_init)
W = DenseTensor(W_init)
m = DenseTensor(m_init)
out = x.dot(W).relu()
out = out.logsoftmax()
out = out.mul(m).add(m).sum()
out.backward()

out.cpu().data, x

WEGHT: (3, 3)


(array([-0.23733956], dtype=float32),
 <DenseTensor <GPUBuffer with shape (1, 3)> with grad <GPUBuffer with shape (1, 3)>>)

In [4]:
x2 = DenseTensor(x2_init)#.gpu()
W = SparseTensor(W_init)
out = W.dot(x2).relu().sum()

out.backward()

out.cpu().data, x

WEGHT: (3,)


IndexError: tuple index out of range

In [5]:
import numpy as np
import pyopencl as cl

mf = cl.mem_flags

In [6]:
dim1 = 8
dim2 = 8
dim3 = 8
bs = 4

ctx = cl.create_some_context()
queue = cl.CommandQueue(ctx,
        properties=cl.command_queue_properties.PROFILING_ENABLE)

sparsity = 0.2

a = np.zeros((dim1,dim2))
b = np.random.rand(4,dim3).astype(np.float32)

a.shape, b.shape

((8, 8), (4, 8))

In [7]:
def fill_sparse(mat, sparsity=0.1):
    indices = np.array(range(mat.shape[1]))
    nrows = int(mat.shape[1]*sparsity)
    for row in range(mat.shape[0]):
        lim = nrows #+ int(np.random.random()*3)
        mat[row][np.random.permutation(indices)[:lim]] = np.random.random(lim)
    return mat

a = fill_sparse(a, 0.9)
#b = fill_sparse(b, sparsity)

In [8]:
a

array([[0.00000000e+00, 4.48052216e-01, 1.43509717e-01, 4.80681472e-01,
        7.52274169e-01, 8.83239468e-02, 3.46555785e-01, 3.19806066e-01],
       [5.83028263e-01, 0.00000000e+00, 9.71479674e-01, 3.99262247e-01,
        9.09366488e-01, 6.15407661e-01, 1.57845720e-01, 3.30874971e-02],
       [0.00000000e+00, 6.47556339e-01, 6.95131742e-01, 9.09672000e-01,
        5.75477319e-02, 5.80649239e-02, 3.20398473e-01, 7.55363370e-01],
       [1.02801017e-01, 8.52259035e-01, 0.00000000e+00, 8.07590847e-01,
        7.29837636e-01, 1.95095816e-01, 9.79401641e-01, 2.12263182e-01],
       [5.72274827e-01, 3.19785390e-02, 0.00000000e+00, 2.33330685e-01,
        7.75220273e-01, 3.09686506e-01, 1.43167083e-01, 4.04916077e-01],
       [7.96191150e-02, 6.77000898e-01, 1.91397334e-01, 1.09839856e-01,
        0.00000000e+00, 2.38284642e-01, 1.58018839e-01, 5.12060840e-01],
       [0.00000000e+00, 1.43816514e-01, 4.70808247e-04, 7.53500277e-01,
        9.00776973e-02, 7.85931198e-01, 9.51000437e-01, 8.

In [9]:
b

array([[0.81256104, 0.22280414, 0.9865953 , 0.85934806, 0.5308143 ,
        0.8673204 , 0.10462095, 0.03061124],
       [0.09537494, 0.19376   , 0.59667635, 0.7977308 , 0.16706866,
        0.9460509 , 0.49244335, 0.46131536],
       [0.14916483, 0.81317675, 0.393102  , 0.6206315 , 0.11614287,
        0.04605395, 0.02824496, 0.64713764],
       [0.48784742, 0.5489166 , 0.16500114, 0.31566837, 0.4405745 ,
        0.87681836, 0.6355691 , 0.41891623]], dtype=float32)

In [10]:
x2_init.T

array([ 1.8688965 , -0.03751023, -0.41621724], dtype=float32)

In [11]:
mult = a.dot(b.T)
mult.shape

(8, 4)

In [12]:
mult.shape

(8, 4)

In [13]:
def to_data(mat):
    all_rows = []
    all_idxs = []
    all_nnzs = []
    for row in range(mat.shape[0]):
        rowdata = []
        colidxs = []
        all_nnzs.append(0)
        for col in range(mat.shape[1]):
            val = mat[row][col]
            if val != 0:
                rowdata.append(val)
                colidxs.append(col)
                all_nnzs[-1] += 1
        all_rows.append(rowdata)
        all_idxs.append(colidxs)
    
    ellwidth = int(np.sqrt(np.max(all_nnzs))+1)**2
    #all_rows = np.array(all_rows)#.astype(np.float32).flatten()           
    for row in range(mat.shape[0]):
        #print(row, all_rows)
        all_rows[row] = np.array(all_rows[row])
        all_rows[row].resize(ellwidth)
        all_idxs[row] = np.array(all_idxs[row])
        all_idxs[row].resize(ellwidth)
        print(all_idxs[row])
    all_rows = np.array(all_rows).astype(np.float32).flatten()
    print(all_idxs)
    all_idxs = np.array(all_idxs).astype(np.uint32).flatten()
    all_nnzs = np.array(all_nnzs).astype(np.uint32)
    
    return all_rows, all_idxs, all_nnzs, ellwidth

In [14]:
adata, acols, annz, ellwa = to_data(a)
adata, acols, annz, ellwa

[1 2 3 4 5 6 7 0 0]
[0 2 3 4 5 6 7 0 0]
[1 2 3 4 5 6 7 0 0]
[0 1 3 4 5 6 7 0 0]
[0 1 3 4 5 6 7 0 0]
[0 1 2 3 5 6 7 0 0]
[1 2 3 4 5 6 7 0 0]
[0 1 2 3 4 6 7 0 0]
[array([1, 2, 3, 4, 5, 6, 7, 0, 0]), array([0, 2, 3, 4, 5, 6, 7, 0, 0]), array([1, 2, 3, 4, 5, 6, 7, 0, 0]), array([0, 1, 3, 4, 5, 6, 7, 0, 0]), array([0, 1, 3, 4, 5, 6, 7, 0, 0]), array([0, 1, 2, 3, 5, 6, 7, 0, 0]), array([1, 2, 3, 4, 5, 6, 7, 0, 0]), array([0, 1, 2, 3, 4, 6, 7, 0, 0])]


(array([4.4805223e-01, 1.4350972e-01, 4.8068148e-01, 7.5227416e-01,
        8.8323943e-02, 3.4655580e-01, 3.1980607e-01, 0.0000000e+00,
        0.0000000e+00, 5.8302826e-01, 9.7147965e-01, 3.9926225e-01,
        9.0936649e-01, 6.1540765e-01, 1.5784572e-01, 3.3087496e-02,
        0.0000000e+00, 0.0000000e+00, 6.4755636e-01, 6.9513172e-01,
        9.0967202e-01, 5.7547733e-02, 5.8064923e-02, 3.2039848e-01,
        7.5536335e-01, 0.0000000e+00, 0.0000000e+00, 1.0280102e-01,
        8.5225904e-01, 8.0759084e-01, 7.2983766e-01, 1.9509582e-01,
        9.7940165e-01, 2.1226318e-01, 0.0000000e+00, 0.0000000e+00,
        5.7227480e-01, 3.1978540e-02, 2.3333068e-01, 7.7522027e-01,
        3.0968651e-01, 1.4316708e-01, 4.0491608e-01, 0.0000000e+00,
        0.0000000e+00, 7.9619117e-02, 6.7700088e-01, 1.9139734e-01,
        1.0983986e-01, 2.3828465e-01, 1.5801884e-01, 5.1206082e-01,
        0.0000000e+00, 0.0000000e+00, 1.4381652e-01, 4.7080824e-04,
        7.5350028e-01, 9.0077698e-02, 7.8593117e

In [15]:
a.shape

(8, 8)

In [16]:
adata.shape, acols.shape, annz.shape, ellwa

((72,), (72,), (8,), 9)

In [17]:
#acols = acols.astype(np.uint32)
#annz = annz.astype(np.uint32)

In [18]:
adata, acols, annz, b

(array([4.4805223e-01, 1.4350972e-01, 4.8068148e-01, 7.5227416e-01,
        8.8323943e-02, 3.4655580e-01, 3.1980607e-01, 0.0000000e+00,
        0.0000000e+00, 5.8302826e-01, 9.7147965e-01, 3.9926225e-01,
        9.0936649e-01, 6.1540765e-01, 1.5784572e-01, 3.3087496e-02,
        0.0000000e+00, 0.0000000e+00, 6.4755636e-01, 6.9513172e-01,
        9.0967202e-01, 5.7547733e-02, 5.8064923e-02, 3.2039848e-01,
        7.5536335e-01, 0.0000000e+00, 0.0000000e+00, 1.0280102e-01,
        8.5225904e-01, 8.0759084e-01, 7.2983766e-01, 1.9509582e-01,
        9.7940165e-01, 2.1226318e-01, 0.0000000e+00, 0.0000000e+00,
        5.7227480e-01, 3.1978540e-02, 2.3333068e-01, 7.7522027e-01,
        3.0968651e-01, 1.4316708e-01, 4.0491608e-01, 0.0000000e+00,
        0.0000000e+00, 7.9619117e-02, 6.7700088e-01, 1.9139734e-01,
        1.0983986e-01, 2.3828465e-01, 1.5801884e-01, 5.1206082e-01,
        0.0000000e+00, 0.0000000e+00, 1.4381652e-01, 4.7080824e-04,
        7.5350028e-01, 9.0077698e-02, 7.8593117e

In [19]:
adata_buf = cl.Buffer(ctx, mf.READ_WRITE | mf.COPY_HOST_PTR, hostbuf=adata)
acols_buf = cl.Buffer(ctx, mf.READ_WRITE | mf.COPY_HOST_PTR, hostbuf=acols)
annzs_buf = cl.Buffer(ctx, mf.READ_WRITE | mf.COPY_HOST_PTR, hostbuf=annz)
b_buf = cl.Buffer(ctx, mf.READ_WRITE | mf.COPY_HOST_PTR, hostbuf=b.T)

prg = cl.Program(ctx, """
// Every global_id_0 works on a row
    __kernel void matmul(__global  float* matData,     // INPUT MATRIX DATA
                            __global  uint*  colIdx,
                            __global  uint*  rowNnz,
                            uint   ellwidth,
                            __global  float* vector_x,    // INPUT
                            __global  float* vector_y    // OUTPUT
                            ) { // LOCAL SHARED BUFFER
      uint gid = get_global_id(0);
      uint nrows = get_global_size(0);
      uint gid2 = get_global_id(1);

      uint nnz    = rowNnz[gid];
      uint baseidx = gid2*nrows;
      float sum = 0;
      for (uint i = 0; i < nnz; i++) {
        uint index   = (gid * ellwidth) + i;
        uint col     = colIdx[index];
        float aval  = matData[index];
        float xval  = vector_x[baseidx+col];
        //printf("aval, xval: %.2f,%.2f: (%i,%i) \\n", aval, xval, col, index);
        sum  += aval * xval;
      }
      printf("SUM/NNZ: %.2f %i \\n", sum, nnz);
      vector_y[baseidx+gid] = sum;
    }""").build()

In [20]:
a.shape, b.shape

((8, 8), (4, 8))

In [21]:
res = np.zeros(a.shape[0]).astype(np.float32)
#res

In [22]:
rows = a.shape[0]

In [23]:
mult = mult.astype(np.float32)

In [24]:
res_buf = cl.Buffer(ctx, mf.WRITE_ONLY, b.nbytes)
knl = prg.matmul  # Use this Kernel object for repeated calls
knl(queue, [rows,bs], None, adata_buf, acols_buf, annzs_buf, np.uint32(ellwa), b_buf, res_buf)

res_np = np.empty_like(b.T)
cl.enqueue_copy(queue, res_np, res_buf)

SUM/NNZ: 1.08 7 
SUM/NNZ: 1.78 7 
SUM/NNZ: 1.84 7 
SUM/NNZ: 1.71 7 
SUM/NNZ: 0.93 7 
SUM/NNZ: 0.88 7 
SUM/NNZ: 2.24 7 
SUM/NNZ: 1.19 7 
SUM/NNZ: 1.18 7 
SUM/NNZ: 1.63 7 
SUM/NNZ: 1.35 7 
SUM/NNZ: 1.98 7 
SUM/NNZ: 1.24 7 
SUM/NNZ: 1.00 7 
SUM/NNZ: 2.00 7 
SUM/NNZ: 0.93 7 
SUM/NNZ: 1.03 7 
SUM/NNZ: 0.88 7 
SUM/NNZ: 1.87 7 
SUM/NNZ: 1.47 7 
SUM/NNZ: 0.63 7 
SUM/NNZ: 1.05 7 
SUM/NNZ: 1.19 7 
SUM/NNZ: 1.10 7 
SUM/NNZ: 1.18 7 
SUM/NNZ: 2.81 7 
SUM/NNZ: 1.75 7 
SUM/NNZ: 1.63 7 
SUM/NNZ: 1.38 7 
SUM/NNZ: 0.74 7 
SUM/NNZ: 1.53 7 
SUM/NNZ: 1.72 7 


In [25]:
res_buf

In [26]:
res_np

array([[1.1764565 , 1.0833288 , 1.0272716 , 1.1844674 ],
       [2.8092957 , 1.7808965 , 0.87648153, 1.6251857 ],
       [1.7493677 , 1.8366982 , 1.8716381 , 1.3536423 ],
       [1.633003  , 1.7059021 , 1.4683648 , 1.9769105 ],
       [1.3801142 , 0.9267018 , 0.6265586 , 1.244091  ],
       [0.7376324 , 0.88006055, 1.0526175 , 1.0005876 ],
       [1.5341926 , 2.235941  , 1.1910982 , 1.9950103 ],
       [1.7229826 , 1.1938562 , 1.1039091 , 0.92708284]], dtype=float32)

In [27]:
mult

array([[1.1764563 , 1.0833288 , 1.0272716 , 1.1844674 ],
       [2.8092957 , 1.7808964 , 0.87648153, 1.6251857 ],
       [1.7493678 , 1.8366982 , 1.8716382 , 1.3536425 ],
       [1.633003  , 1.7059021 , 1.4683648 , 1.9769106 ],
       [1.3801141 , 0.9267018 , 0.62655866, 1.244091  ],
       [0.7376324 , 0.8800605 , 1.0526175 , 1.0005876 ],
       [1.5341926 , 2.235941  , 1.1910983 , 1.9950103 ],
       [1.7229828 , 1.1938561 , 1.1039091 , 0.92708284]], dtype=float32)

In [28]:
res_np.shape

(8, 4)

In [29]:
mult.shape

(8, 4)

In [30]:
(res_np-mult).sum()

-2.3841858e-07

## Weight update kernel

In [464]:
bs = 4

In [478]:
dim = 512
topk = 2

x = np.random.rand(bs,dim).astype(np.float32)
y = np.random.rand(bs,dim).astype(np.float32)
x.shape,y.shape, topk

((4, 512), (4, 512), 2)

In [479]:
x_buf = cl.Buffer(ctx, mf.READ_ONLY | mf.COPY_HOST_PTR, hostbuf=x)
y_buf = cl.Buffer(ctx, mf.READ_ONLY | mf.COPY_HOST_PTR, hostbuf=y)
x_cp_buf = cl.Buffer(ctx, mf.WRITE_ONLY, bs*topk*topk*4)
x_idx_buf = cl.Buffer(ctx, mf.WRITE_ONLY, bs*topk*4)
y_idx_buf = cl.Buffer(ctx, mf.WRITE_ONLY, bs*topk*4)

prg = cl.Program(ctx, """
    // sorts x and y in ascending order and returns sorted indices
    __kernel void genwupdate2(__global  float* x,     // INPUT MATRIX DATA
                             __global  float* y,    // INPUT
                             __global  float* xout,    // INPUT
                             uint topk,
                             __global  uint* xoutidx,    // INPUT
                             __global  uint* youtidx    // INPUT
                            ) { // LOCAL SHARED BUFFER
      uint gid = get_global_id(0);
      uint n = get_global_size(0);
      uint bs = get_global_size(1);
      uint gid2 = get_global_id(1);

      uint idx = n*gid2+gid;

      float valx = x[idx];
      float valy = y[idx];
      uint posx = 0;
      uint posy = 0;
      for (uint i = 0; i < n; i++) {
        uint idx2 = n*gid2+i;
        float tempval = x[idx2];
        float tempval2 = y[idx2];
        bool larger = tempval > valx;
        bool larger2 = tempval2 > valy;

        posx += (larger)?1:0;
        posy += (larger2)?1:0;
      }
      //printf("posx:%i", posx);
      if (posx < topk) {
        xoutidx[posx+topk*gid2] = gid;
      }
      if (posy < topk) {
        youtidx[posy+topk*gid2] = gid;
      }
      if (gid < topk) {
        for (uint j=0; j<topk; j++) {
          float res = x[xoutidx[gid+topk*gid2]+gid2*n] * y[youtidx[j+topk*gid2]+gid2*n];
          printf("\\nRES:%.2f - %i - %i -  %.2f - %.2f",res, xoutidx[gid]+topk*gid2, youtidx[j]+topk*gid2, x[xoutidx[gid]+n*gid2], y[youtidx[j]+n*gid2]);
          xout[gid2*topk*topk+j*topk+gid] = res;
        }
      }
    }""").build()

In [480]:
knl = prg.genwupdate2  # Use this Kernel object for repeated calls
knl(queue, [dim,bs], None, x_buf, y_buf, x_cp_buf, np.uint32(topk), x_idx_buf, y_idx_buf)

resx = np.zeros(bs*topk*topk).astype(np.float32)
resxidx = np.zeros(bs*topk).astype(np.uint32)
resyidx = np.zeros(bs*topk).astype(np.uint32)

cl.enqueue_copy(queue, resx, x_cp_buf)
cl.enqueue_copy(queue, resxidx, x_idx_buf)
cl.enqueue_copy(queue, resyidx, y_idx_buf)


RES:1.00 - 130 - 357 -  0.03 - 0.78
RES:0.89 - 231 - 357 -  0.91 - 0.78
RES:0.83 - 126 - 353 -  0.87 - 0.95
RES:0.95 - 227 - 353 -  0.99 - 0.95
RES:0.47 - 180 - 187 -  0.04 - 0.41
RES:0.47 - 233 - 187 -  0.19 - 0.41
RES:0.99 - 128 - 355 -  0.36 - 0.24
RES:0.99 - 229 - 355 -  0.89 - 0.24
RES:0.99 - 178 - 451 -  0.37 - 0.35
RES:0.99 - 231 - 451 -  0.91 - 0.35
RES:1.00 - 180 - 453 -  0.04 - 0.39
RES:0.99 - 233 - 453 -  0.19 - 0.39
RES:0.99 - 174 - 447 -  1.00 - 1.00
RES:0.99 - 227 - 447 -  0.99 - 1.00
RES:0.99 - 176 - 449 -  0.72 - 0.20
RES:0.99 - 229 - 449 -  0.89 - 0.20

In [481]:
x

array([[0.46777302, 0.20140186, 0.18767795, ..., 0.537703  , 0.19379517,
        0.6404819 ],
       [0.3358809 , 0.62642974, 0.5079018 , ..., 0.5344909 , 0.60313153,
        0.7063903 ],
       [0.5627141 , 0.52585715, 0.628985  , ..., 0.614591  , 0.63342434,
        0.03479806],
       [0.3106645 , 0.02247608, 0.45861515, ..., 0.6346314 , 0.03360393,
        0.5066719 ]], dtype=float32)

In [482]:
y

array([[0.9369933 , 0.89566046, 0.06341496, ..., 0.14309883, 0.44046733,
        0.41868305],
       [0.96686137, 0.21332563, 0.249524  , ..., 0.7894286 , 0.72005105,
        0.7040048 ],
       [0.6535747 , 0.24967723, 0.8565109 , ..., 0.65692794, 0.08650134,
        0.4061329 ],
       [0.67563355, 0.899449  , 0.6795114 , ..., 0.5462706 , 0.3270566 ,
        0.48116744]], dtype=float32)

In [483]:
x.shape, y.shape

((4, 512), (4, 512))

In [484]:
resx

array([0.8324195 , 0.9467327 , 0.9944083 , 0.9901414 , 0.9932806 ,
       0.99224687, 0.99266356, 0.99163043, 0.99556315, 0.8933518 ,
       0.9910823 , 0.98543465, 0.47427627, 0.4736977 , 0.99600077,
       0.99478567], dtype=float32)

In [485]:
resx.reshape(bs,topk,topk)

array([[[0.8324195 , 0.9467327 ],
        [0.9944083 , 0.9901414 ]],

       [[0.9932806 , 0.99224687],
        [0.99266356, 0.99163043]],

       [[0.99556315, 0.8933518 ],
        [0.9910823 , 0.98543465]],

       [[0.47427627, 0.4736977 ],
        [0.99600077, 0.99478567]]], dtype=float32)

In [486]:
resxidx

array([174, 227,   3, 409, 158, 383,  11,  30], dtype=uint32)

In [487]:
resyidx

array([181, 447, 392,  68, 426, 430, 285, 161], dtype=uint32)

In [488]:
idx = 0
xy0 = x[idx].reshape(dim,1)*y[idx]
xy0.shape

(512, 512)

In [489]:
xy0[126][353]

0.8324195

## Weight update kernel

dim = 16
topk = 4

x = np.random.rand(dim).astype(np.float32)
y = np.random.rand(dim).astype(np.float32)
x.shape,y.shape

In [ ]:
#x_buf = cl.Buffer(ctx, mf.READ_ONLY | mf.COPY_HOST_PTR, hostbuf=x)
#y_buf = cl.Buffer(ctx, mf.READ_ONLY | mf.COPY_HOST_PTR, hostbuf=y)
#x_cp_buf = cl.Buffer(ctx, mf.WRITE_ONLY, 4*topk*topk)
#x_idx_buf = cl.Buffer(ctx, mf.WRITE_ONLY, 4*topk)
#y_idx_buf = cl.Buffer(ctx, mf.WRITE_ONLY, 4*topk)

prg = cl.Program(ctx, """
// Every global_id_0 works on a row
__kernel void genwupdate2(__global  float* x,     // INPUT MATRIX DATA
                             __global  float* y,    // INPUT
                             __global  float* xout,    // INPUT
                             uint topk,
                             __global  uint* xoutidx,    // INPUT
                             __global  uint* youtidx    // INPUT
                            ) { // LOCAL SHARED BUFFER
      uint gid = get_global_id(0);
      uint n = get_global_size(0);

      xout[gid] = x[gid];
      xoutidx[gid] = gid;
      youtidx[gid] = gid;

      float valx = x[gid];
      float valy = y[gid];
      uint posx = 0;
      uint posy = 0;
      for (uint i = 0; i < n; i++) {
        float tempval = x[i];
        float tempval2 = y[i];
        bool larger = tempval > valx;
        bool larger2 = tempval2 > valy;

        posx += (larger)?1:0;
        posy += (larger2)?1:0;
      }
      //printf("posx:%i", posx);
      if (posx < topk) {
        xoutidx[posx] = gid;
      }
      if (posy < topk) {
        youtidx[posy] = gid;
      }
      if (gid < topk) {
        uint i = gid;
        for (uint j=0; j<topk; j++) {
          xout[gid*topk+j] = x[xoutidx[gid]] * y[youtidx[j]];
        }
      }
    }""").build()

In [ ]:
knl = prg.genwupdate2  # Use this Kernel object for repeated calls
knl(queue, [dim,], None, x_buf, y_buf, x_cp_buf, np.uint32(topk), x_idx_buf, y_idx_buf)

resx = np.zeros(dim).astype(np.float32)
resxidx = np.zeros(topk).astype(np.uint32)
resyidx = np.zeros(topk).astype(np.uint32)

cl.enqueue_copy(queue, resx, x_cp_buf)
cl.enqueue_copy(queue, resxidx, x_idx_buf)
cl.enqueue_copy(queue, resyidx, y_idx_buf)

In [ ]:
x

In [ ]:
resx

In [ ]:
resx.reshape(4,4)

In [ ]:
resxidx

In [ ]:
resyidx

In [ ]:
x*y

In [ ]:
x.reshape(dim,1)*y[7]

In [ ]:
y

In [ ]:
resy

In [ ]:
resyidx

### update vals

In [ ]:
prg = cl.Program(ctx, """
// Every global_id_0 works on a row
__kernel void addvals(__global  float* matData,     // INPUT MATRIX DATA
                         __global  uint*  colIdx,
                         __global  uint*  rowNnz,
                         uint   ellwidth,
                         __global  float* updatevals,    // INPUT
                         __global  uint* updatexidx,   
                         __global  uint* updateyidx   
                         ) { // LOCAL SHARED BUFFER
      uint gid = get_global_id(0);
      uint n = get_global_size(0);
      uint row = updateyidx[gid];
      
      for (uint i=0; i<n; i++) {
        float val = updatevals[gid*n+i];
        uint col = updatexidx[i];
        for (uint i=0; i<rowNnz[row]; i++) {
          uint idx = gid*ellwidth+i;
          if (colIdx[idx] >= col) {
            if (colIdx[idx] == col) {
              matData[idx] += val;
              printf("UPDATE[%i]: %.2f", idx, val);
              break;
            } else {
              // insert new column
              printf("INSERT[%i]: %.2f", idx, val);
              for (uint j=rowNnz[row]+1; j>i; j--) {
                uint idx2 = gid*ellwidth+j;
                matData[idx2] = matData[idx2-1];
                colIdx[idx2] = colIdx[idx2-1];
              }
              matData[idx] = val;
              colIdx[idx] = col;
              rowNnz[row] += 1;
              break;
            }
          }
        }
        if (rowNnz[row] >= ellwidth) {
          break;
        }
      }

      
    }""").build()

In [ ]:
knl = prg.addvals  # Use this Kernel object for repeated calls
knl(queue, [topk,], None, adata_buf, acols_buf, annzs_buf, np.uint32(ellwa), x_cp_buf, x_idx_buf, y_idx_buf)

resa = np.empty_like(adata)
resaidx = np.zeros(acols.shape).astype(np.uint32)
resannz = np.zeros(annz.shape).astype(np.uint32)

cl.enqueue_copy(queue, resa, adata_buf)
cl.enqueue_copy(queue, resaidx, acols_buf)
cl.enqueue_copy(queue, resannz, annzs_buf)

In [ ]:
resa

In [ ]:
adata

In [ ]:
acols

In [ ]:
resaidx

In [ ]:
resannz

In [ ]:
asdf

# OTHER

import numpy as np
import pyopencl as cl

mf = cl.mem_flags

dim = 16
topk = 4

x = np.random.rand(dim).astype(np.float32)
y = np.random.rand(dim).astype(np.float32)
x.shape,y.shape

dim1 = 4
dim2 = 8
dim3 = 1

ctx = cl.create_some_context()
queue = cl.CommandQueue(ctx,
        properties=cl.command_queue_properties.PROFILING_ENABLE)

sparsity = 0.2

a = np.zeros((dim1,dim2))
b = np.random.rand(dim2,dim3).flatten().astype(np.float32)

a.shape, b.shape

In [ ]:
x_buf = cl.Buffer(ctx, mf.READ_ONLY | mf.COPY_HOST_PTR, hostbuf=x)
y_buf = cl.Buffer(ctx, mf.READ_ONLY | mf.COPY_HOST_PTR, hostbuf=y)
val_out_buf = cl.Buffer(ctx, mf.READ_WRITE, 4*topk*topk)
x_idx_buf = cl.Buffer(ctx, mf.READ_WRITE, topk*4)
y_idx_buf = cl.Buffer(ctx, mf.READ_WRITE, topk*4)

prg = cl.Program(ctx, """
// Every global_id_0 works on a row
__kernel void genwupdate2(__global  float* x,     // INPUT MATRIX DATA
                         __global  float* y,    // INPUT
                         __global  float* xout,    // INPUT
                         uint topk,
                         __global  uint* xoutidx,    // INPUT
                         __global  uint* youtidx    // INPUT
                        ) { // LOCAL SHARED BUFFER
  uint gid = get_global_id(0);
  uint n = get_global_size(0);
  
  xout[gid] = x[gid];
  xoutidx[gid] = gid;
  youtidx[gid] = gid;
  
  float valx = x[gid];
  float valy = y[gid];
  uint posx = 0;
  uint posy = 0;
  for (uint i = 0; i < n; i++) {
    float tempval = x[i];
    float tempval2 = y[i];
    bool larger = tempval > valx;
    bool larger2 = tempval2 > valy;
      
    posx += (larger)?1:0;
    posy += (larger2)?1:0;
  }
  //printf("posx:%i", posx);
  if (posx < topk) {
    xoutidx[posx] = gid;
  }
  if (posy < topk) {
    youtidx[posy] = gid;
  }
  if (gid < topk) {
    uint i = gid;
    for (uint j=0; j<topk; j++) {
      xout[gid*topk+j] = x[xoutidx[gid]] * y[youtidx[j]];
    }
  }
}""").build()

In [ ]:
knl = prg.genwupdate2  # Use this Kernel object for repeated calls
event = knl(queue, [dim,], None, x_buf, y_buf, val_out_buf, np.uint32(topk), x_idx_buf, y_idx_buf)

#event.wait()
val_out = np.zeros(topk*topk).astype(np.float32)
resxidx = np.zeros(topk).astype(np.uint32)
resyidx = np.zeros(topk).astype(np.uint32)

cl.enqueue_copy(queue, val_out, val_out_buf)
cl.enqueue_copy(queue, resxidx, x_idx_buf, wait_for=[event])
cl.enqueue_copy(queue, resyidx, y_idx_buf)

In [ ]:
val_out

In [ ]:
resxidx

In [ ]:
resyidx

In [ ]:
from __future__ import division

KERNEL_CODE = """
// Thread block size
#define BLOCK_SIZE %(block_size)d
// Matrix dimensions
// (chosen as multiples of the thread block size for simplicity)
#define WA %(w_a)d // Matrix A width
#define HA %(h_a)d // Matrix A height
#define WB %(w_b)d // Matrix B width
#define HB WA  // Matrix B height
#define WC WB  // Matrix C width
#define HC HA  // Matrix C height
/*
 * Copyright 1993-2009 NVIDIA Corporation.  All rights reserved.
 *
 * NVIDIA Corporation and its licensors retain all intellectual property and
 * proprietary rights in and to this software and related documentation.
 * Any use, reproduction, disclosure, or distribution of this software
 * and related documentation without an express license agreement from
 * NVIDIA Corporation is strictly prohibited.
 *
 * Please refer to the applicable NVIDIA end user license agreement (EULA)
 * associated with this source code for terms and conditions that govern
 * your use of this NVIDIA software.
 *
 */
/* Matrix multiplication: C = A * B.
 * Device code.
 */
#define AS(j, i) As[i + j * BLOCK_SIZE]
#define BS(j, i) Bs[i + j * BLOCK_SIZE]
////////////////////////////////////////////////////////////////////////////////
//! Matrix multiplication on the device: C = A * B
//! WA is A's width and WB is B's width
////////////////////////////////////////////////////////////////////////////////
__kernel __attribute__((reqd_work_group_size(16,16,1))) 
void
matrixMul( __global float* C, __global float* A, __global float* B)
{
    __local float As[BLOCK_SIZE*BLOCK_SIZE];
    __local float Bs[BLOCK_SIZE*BLOCK_SIZE];
    // Block index
    int bx = get_group_id(0);
    int by = get_group_id(1);
    // Thread index
    int tx = get_local_id(0);
    int ty = get_local_id(1);
    // Index of the first sub-matrix of A processed by the block
    int aBegin = WA * BLOCK_SIZE * by;
    // Index of the last sub-matrix of A processed by the block
    int aEnd   = aBegin + WA - 1;
    // Step size used to iterate through the sub-matrices of A
    int aStep  = BLOCK_SIZE;
    // Index of the first sub-matrix of B processed by the block
    int bBegin = BLOCK_SIZE * bx;
    // Step size used to iterate through the sub-matrices of B
    int bStep  = BLOCK_SIZE * WB;
    // Csub is used to store the element of the block sub-matrix
    // that is computed by the thread
    float Csub = 0.0f;
    // Loop over all the sub-matrices of A and B
    // required to compute the block sub-matrix
    for (int a = aBegin, b = bBegin;
             a <= aEnd;
             a += aStep, b += bStep) {
        // Load the matrices from device memory
        // to shared memory; each thread loads
        // one element of each matrix
        AS(ty, tx) = A[a + WA * ty + tx];
        BS(ty, tx) = B[b + WB * ty + tx];
        // Synchronize to make sure the matrices are loaded
        barrier(CLK_LOCAL_MEM_FENCE);
        // Multiply the two matrices together;
        // each thread computes one element
        // of the block sub-matrix
        for (int k = 0; k < BLOCK_SIZE; ++k)
            Csub += AS(ty, k) * BS(k, tx);
        // Synchronize to make sure that the preceding
        // computation is done before loading two new
        // sub-matrices of A and B in the next iteration
        barrier(CLK_LOCAL_MEM_FENCE);
    }
    // Write the block sub-matrix to device memory;
    // each thread writes one element
    C[get_global_id(1) * get_global_size(0) + get_global_id(0)] = Csub;
}
"""
